In [ ]:
# dataframe
import pandas as pd

df = pd.read_csv('./payload/train.csv')

In [18]:
# load all models from models/

import os
import pickle

models = []
for filename in os.listdir('./models'):
    # load pth
    if filename.endswith('.pth'):
        with open(os.path.join('./models', filename), 'rb') as f:
            model = pickle.load(f)
            models.append(model)


In [ ]:
paths = [
    'history\\runs\\run_vit_base_20241207_113554_ep28\\',
    'history\\runs\\run_efficientnet_20241205_114438_ep7\\',
    'history\\runs\\run_resnet50_20241208_231720_ep6\\',
    'history\\runs\\run_resnet101_20241204_201138_ep14\\',
    'history\\runs\\run_vit_mediumd_20241209_002034_ep26\\',
]

In [43]:
# get all models in models/ 
import os


models = []
for filename in os.listdir('./models'):
    # load pth
    if filename.endswith('.pth'):
        models.append(filename)

# get all runs in history/runs

runs = []
for filename in os.listdir('./history/runs'):
    runs.append(filename)

# match models with runs

matched_models = []
for model in models:
    for run in runs:
        if model.split('_')[1] in run:
            matched_models.append((model, run))

# print matched models

for model, run in matched_models:
    print(f'Model: {model}, Run: {run}')



Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180702
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180709
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180714
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180718
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180723
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180920
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180926
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180931
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficientnet_20241204_180935
Model: run_efficientnet_20241205_114438_ep7_best_auc_model.pth, Run: run_efficient